<a href="https://colab.research.google.com/github/advait-t/IPL_Datasets/blob/main/IPL_2021_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IPL 2021

In [1]:
import datetime
from datetime import date, timedelta
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import pytz
from datetime import datetime, date, time

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
tz_in = pytz.timezone('Asia/Calcutta') 
datetime_in = datetime.now(tz_in)
times = int(datetime_in.strftime("%H"))

print("India time:", times)

India time: 9


In [4]:
def season_stats():
    sid=[1249214] #each season id on ESPN Cricinfo
    mid=[1254058] #1st MatchID of each season
    season = [14]
    matches = 1
    start = date(2021, 4, 9)
    if times >= 19 and times <= 00:
        today = date.today()
        matches = matches - 1
    elif times >= 00
        
    else:
        today = date.today() - timedelta(1)
    double = [date(2021, 4, 18),date(2021, 4, 21),date(2021, 4, 29),date(2021, 5, 2),date(2021, 5, 8),date(2021, 5, 9),date(2021, 5, 13),date(2021, 5, 16),date(2021, 5, 21),date(2021, 5, 23)]
    for i in range(0,len(double)):
        if ((int((double[i]-start).days)) <= (int((today-start).days))):
            matches = (matches + 1)
        else:
            pass
    matches = (matches + int((today-start).days))
    return matches,season,sid,mid

SyntaxError: ignored

In [5]:
season_stats()

NameError: ignored

In [ ]:
def extract_batting_data(series_id, match_id,season_id):

    URL = 'https://www.espncricinfo.com/series/'+ str(series_id) + '/scorecard/' + str(match_id)
    page = requests.get(URL)
    try:
        bs = BeautifulSoup(page.content, 'lxml')
        table_body=bs.find_all('tbody')
        batsmen_df = pd.DataFrame(columns=["Name","Description","Runs", "Balls", "4s", "6s", "SR", "Team","Match_id","Season"])
        for i, table in enumerate(table_body[0:4:2]):
            rows = table.find_all('tr')
            for row in rows[::2]:
                cols=row.find_all('td')
                cols=[x.text.strip() for x in cols]
                if cols[0] == 'Extras':
                    continue
                if len(cols) > 7:
                    batsmen_df = batsmen_df.append(pd.Series(
                    [re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip(), cols[1], 
                    cols[2], cols[3], cols[5], cols[6], cols[7],i+1,match_id,season_id], 
                    index=batsmen_df.columns ), ignore_index=True)
                else:
                    batsmen_df = batsmen_df.append(pd.Series(
                    [re.sub(r"\W+", ' ', cols[0].split("(c)")[0]).strip(), cols[1], 
                    0, 0, 0, 0, 0, i+1], index = batsmen_df.columns), ignore_index=True)
    except:
        pass 
    return batsmen_df

In [ ]:
def batting_csv(matches,season,sid,mid):
    batting_data1=[]
    for n,i in enumerate(season):
        if (sid[n]==1249214):
            a = matches
        for j in range(mid[n],mid[n]+a):
            #print(i,j)
            x=extract_batting_data(series_id = i, match_id = j,season_id=i)
            batting_data1.append(x)
            batting_data=pd.concat(batting_data1)
    batting_data.to_csv('/content/batting_data_IPL14.csv')

In [ ]:
def extract_bowling_data(series_id, match_id,season_id):

    URL = 'https://www.espncricinfo.com/series/'+ str(series_id) + '/scorecard/' + str(match_id)
    page = requests.get(URL)
    try:
        bs = BeautifulSoup(page.content, 'lxml')

        table_body=bs.find_all('tbody')
        bowler_df = pd.DataFrame(columns=['Name', 'Overs', 'Maidens', 'Runs', 'Wickets',
                                        'Econ', 'Dots', '4s', '6s', 'Wd', 'Nb','Team','Match_id','Season'])
        for i, table in enumerate(table_body[1:4:2]):
            rows = table.find_all('tr')
            for row in rows:
                cols=row.find_all('td')
                cols=[x.text.strip() for x in cols]
                bowler_df = bowler_df.append(pd.Series([cols[0], cols[1], cols[2], cols[3], cols[4], cols[5], 
                                                        cols[6], cols[7], cols[8], cols[9], cols[10], (i==0)+1,match_id,season_id], 
                                                    index=bowler_df.columns ), ignore_index=True)
    except:
        pass
    return bowler_df

In [ ]:
def bowling_csv(matches,season,sid,mid):
    bowling_data1=[]
    for n,i in enumerate(season):
        if (sid[n]==1249214):
            a = matches
        for j in range(mid[n],mid[n]+a):
            #print(i,j)
            x=extract_bowling_data(series_id = i, match_id = j,season_id=i)
            bowling_data1.append(x)
            bowling_data=pd.concat(bowling_data1)
            bowling_data.to_csv('/content/bowling_data_IPL14.csv')

In [ ]:
matches, season, sid, mid = season_stats()
batting_csv(matches, season, sid, mid)
bowling_csv(matches, season, sid, mid)

# Fantasy Points Function

In [ ]:
#Batting

In [ ]:
def bat_fantasy(batting_data):
    convert_dict = {'Runs': int,
                    'Balls':int,
                    '4s': int,
                    '6s': int,
                    'SR': float
                }
    
    batting_data = batting_data.astype(convert_dict)
    bat_fantasy_cal(batting_data)

In [ ]:
def bat_fantasy_cal(batting_data):

    batting_data['Fantasy_Points'] = batting_data['Runs']

    for i in range(len(batting_data['4s'])):
        if (batting_data.iloc[i]['4s'] >= 1):
            batting_data['Fantasy_Points'][i] = batting_data.iloc[i]['Fantasy_Points']+batting_data.iloc[i]['4s']
            
    for j in range(len(batting_data['6s'])):
        if (batting_data.iloc[j]['6s'] >= 1):
            batting_data['Fantasy_Points'][j] = batting_data.iloc[j]['Fantasy_Points']+(2*(batting_data.iloc[j]['6s']))

    for k in range(len(batting_data['Runs'])):
        if (batting_data.iloc[k]['Runs'] >= 30 and batting_data.iloc[k]['Runs'] < 50):
            batting_data['Fantasy_Points'][k] = batting_data.iloc[k]['Fantasy_Points']+4
        elif (batting_data.iloc[k]['Runs'] >= 50 and batting_data.iloc[k]['Runs'] < 100):
            batting_data['Fantasy_Points'][k] = batting_data.iloc[k]['Fantasy_Points']+12
        elif (batting_data.iloc[k]['Runs'] >= 100):
            batting_data['Fantasy_Points'][k] = batting_data.iloc[k]['Fantasy_Points']+16
        elif (batting_data.iloc[k]['Runs'] == 0):
            batting_data['Fantasy_Points'][k] = batting_data.iloc[k]['Fantasy_Points']-2

    for l in range(len(batting_data['Balls'])):
        if (batting_data.iloc[l]['Balls']>=10):
            if (batting_data.iloc[l]['SR'] >= 170):
                batting_data['Fantasy_Points'][l] = batting_data.iloc[l]['Fantasy_Points']+6

            elif (batting_data.iloc[l]['SR'] >= 150.01 and batting_data.iloc[l]['SR'] < 170):
                batting_data['Fantasy_Points'][l] = batting_data.iloc[l]['Fantasy_Points']+4

            elif (batting_data.iloc[l]['SR'] >= 130 and batting_data.iloc[l]['SR'] <= 150):
                batting_data['Fantasy_Points'][l] = batting_data.iloc[l]['Fantasy_Points']+2

            elif (batting_data.iloc[l]['SR'] >= 60 and batting_data.iloc[l]['SR'] <= 70):
                batting_data['Fantasy_Points'][l] = batting_data.iloc[l]['Fantasy_Points']-2

            elif (batting_data.iloc[l]['SR'] >= 50 and batting_data.iloc[l]['SR'] <= 59.99):
                batting_data['Fantasy_Points'][l] = batting_data.iloc[l]['Fantasy_Points']-4

            elif (batting_data.iloc[l]['SR'] < 50):
                batting_data['Fantasy_Points'][l] = batting_data.iloc[l]['Fantasy_Points']-6
    return batting_data
    #batting_data.to_csv('/content/batting_data_IPL14.csv')

In [ ]:
batting_data = pd.read_csv('/content/batting_data_IPL14.csv')
batting_data = bat_fantasy(batting_data)

In [ ]:
#Bowling

In [ ]:
def bowl_fantasy(bowling_data):
    convert_dict = {'Runs': int,
                    'Maidens':int,
                    'Overs': float,
                    '6s': int,
                    '4s': int,
                    'Wd': int,
                    'Nb': int,
                    'Dots': int,
                    'Econ': float,
                    'Wickets': int
                }
    
    bowling_data = bowling_data.astype(convert_dict)
    bowl_fantasy_cal(bowling_data)

In [ ]:
def bowl_fantasy_cal(bowling_data):
    bowling_data['Fantasy_Points'] = 0
    for i in range(len(bowling_data['Wickets'])):
        if (bowling_data.iloc[i]['Wickets'] >= 1):
            bowling_data['Fantasy_Points'][i] = bowling_data.iloc[i]['Wickets']*25

            if (bowling_data.iloc[i]['Wickets'] == 3):
                bowling_data['Fantasy_Points'][i] = bowling_data.iloc[i]['Fantasy_Points'] + 4
            elif (bowling_data.iloc[i]['Wickets'] == 4):
                bowling_data['Fantasy_Points'][i] = bowling_data.iloc[i]['Fantasy_Points'] + 8
            elif (bowling_data.iloc[i]['Wickets'] == 5):
                bowling_data['Fantasy_Points'][i] = bowling_data.iloc[i]['Fantasy_Points'] + 16

    for j in range(len(bowling_data['Maidens'])):
        if (bowling_data.iloc[j]['Maidens'] >= 1):
            bowling_data['Fantasy_Points'][j] = bowling_data.iloc[j]['Fantasy_Points'] + (bowling_data.iloc[j]['Maidens']*12)

    for k in range(len(bowling_data['Overs'])):

        if (bowling_data.iloc[k]['Overs'] >= 2):

            if (bowling_data.iloc[j]['Econ'] < 5.0):
                bowling_data['Fantasy_Points'][k] = bowling_data.iloc[k]['Fantasy_Points'] + 6

            elif (bowling_data.iloc[j]['Econ'] >= 5.0 and bowling_data.iloc[j]['Econ'] <= 5.99):
                bowling_data['Fantasy_Points'][k] = bowling_data.iloc[k]['Fantasy_Points'] + 4

            elif (bowling_data.iloc[j]['Econ'] >= 6.0 and bowling_data.iloc[j]['Econ'] <= 7.0):
                bowling_data['Fantasy_Points'][k] = bowling_data.iloc[k]['Fantasy_Points'] + 2

            elif (bowling_data.iloc[j]['Econ'] >= 10.0 and bowling_data.iloc[j]['Econ'] <= 11.0):
                bowling_data['Fantasy_Points'][k] = bowling_data.iloc[k]['Fantasy_Points'] - 2

            elif (bowling_data.iloc[j]['Econ'] >= 11.01 and bowling_data.iloc[j]['Econ'] <= 12.0):
                bowling_data['Fantasy_Points'][k] = bowling_data.iloc[k]['Fantasy_Points'] - 4

            elif (bowling_data.iloc[j]['Econ'] >= 12.0):
                bowling_data['Fantasy_Points'][k] = bowling_data.iloc[k]['Fantasy_Points'] - 6
    return bowling_data
    #bowling_data.to_csv('/content/bowling_data_IPL14.csv')

In [ ]:
bowling_data = pd.read_csv('/content/bowling_data_IPL14.csv')
bowling_data = bowl_fantasy(bowling_data)